# RISS에서 서지정보 자동 내보내기

## 00 라이브러리

In [1]:
#크롬 버전 확인 후 크롬 드라이버 설치 필요. 크롬 드라이버 다운로드 받은 파일 경로 기억해둘 것.

In [3]:
# 필요 라이브러리 설치
!pip install pandas selenium tqdm beautifulsoup4 requests
#!apt-get update # to update ubuntu to correctly run apt install
#!apt install chromium-chromedriver

# 라이브러리 불러오기
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from tqdm import tqdm
import time, os, shutil, requests #시간, 시스템, 셸유틸리티(파일명 바꾸기), http 요청

from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import UnexpectedAlertPresentException

#### 다운로드 폴더 설정(riss 폴더)

In [8]:
# download_path = os.getcwd()
download_path = '/Users/LENOVO/Desktop/riss_test'
download_path

'/Users/LENOVO/Desktop/riss_test'

In [9]:
# 참고 : https://cozynow.tistory.com/43
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
  "download.default_directory": download_path,
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})

## 01 크롬 가동 및 셀레늄 시험
아래 링크에서 운영체제에 맞는 크롬 드라이버 다운로드
https://chromedriver.chromium.org/downloads

In [11]:
driver = webdriver.Chrome('/Users/LENOVO/Downloads/chromedriver', options=options) #chromedriver 설치 위치 지정
#driver.implicitly_wait(3) #웹페이지 로드를 기다리는 시간을 지정할 수 있음 괄호 안에 숫자로 지정

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_101008\3678066498.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/LENOVO/Downloads/chromedriver', options=options) #chromedriver 설치 위치 지정


#### 논문 검색조건
출판년도 : 2000~2019  
학과정보 : '국어국문' OR '국문'  
학위 : 국내박사

In [12]:
driver.get('https://www-riss-kr-ssl.glibproxy.gachon.ac.kr:8443/search/Search.do?isDetailSearch=Y&searchGubun=true&viewYn=OP&queryText=znTitle%2C%EC%A1%B0%EC%84%A0%EC%A1%B1%40op%2COR%40znAbstract%2C%EC%A1%B0%EC%84%A0%EC%A1%B1&strQuery=&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=1992&p_year2=2002&iStartCount=0&orderBy=&mat_type=&mat_subtype=T2%2CT1&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=')

In [13]:
# 파일선택
# 문법 바뀜 by 이하 삭제
#driver.find_element(By.CSS_SELECTOR, '#divContent > div > div.rightContent.wd756 > div > div.srchResultW > div.srchResultListW > ul > li:nth-child(1) > div.cont.ml60 > p.title > a').click() #파일 하나
driver.find_element(By.CSS_SELECTOR, '#divContent > div > div.rightContent.wd756 > div > div.srchResultW > div:nth-child(1) > div.resultTop1 > label > span').click()
#divContent > div > div.rightContent.wd756 > div > div.srchResultW > div:nth-child(1) > div.resultTop1 > label > span

In [16]:
# 내보내기
# #thesisInfoDiv > div.btnBunch > div.btnBunchR > ul > li:nth-child(1) > a
# #divContent > div > div.rightContent.wd756 > div > div.srchResultW > div:nth-child(1) > div.resultTop1 > ul > li:nth-child(1) > a
# driver.find_element(By.CSS_SELECTOR, '#thesisInfoDiv > div.btnBunch > div.btnBunchR > ul > li:nth-child(1) > a').click()
driver.find_element(By.CSS_SELECTOR, '#divContent > div > div.rightContent.wd756 > div > div.srchResultW > div:nth-child(1) > div.resultTop1 > ul > li:nth-child(1) > a').click()

In [17]:
# 팝업창 컨트롤
driver.switch_to.window(driver.window_handles[-1])

In [18]:
# Excel 저장 선택
driver.find_element(By.CSS_SELECTOR, '#wrap > form > div > div.popCont > div:nth-child(1) > div > ul > li:nth-child(3) > label').click()

In [19]:
# Excel 상세정보 선택
driver.find_element(By.CSS_SELECTOR, '#riss_gubun > ul > li:nth-child(2) > label').click()

In [20]:
# 데이터 표시양식 '가로' 설정
driver.find_element(By.CSS_SELECTOR, '#excel_gubun > ul > li:nth-child(2) > label').click()

In [21]:
# 내보내기 클릭
driver.find_element(By.CSS_SELECTOR, '#riss_gubun > div.btnBunch > a.btnType1').click()

In [22]:
# 다시 메인 윈도우로 복귀
driver.switch_to.window(driver.window_handles[0])

## 02 자동 내보내기

#### RISS에서 가장 마지막 페이지 번호

In [30]:
n = 2

#### 자동 다운로드 파일 이름 설정

In [31]:
keyword = '조선족'

#### 자동 내보내기 시작

In [32]:
driver = webdriver.Chrome('/Users/LENOVO/Downloads/chromedriver', options=options) #chromedriver 설치 위치 지정

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_101008\3564001305.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/LENOVO/Downloads/chromedriver', options=options) #chromedriver 설치 위치 지정


In [35]:
driver.get('https://www-riss-kr-ssl.glibproxy.gachon.ac.kr:8443/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%A1%B0%EC%84%A0%EC%A1%B1&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=%EC%A1%B0%EC%84%A0%EC%A1%B1&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EC%A1%B0%EC%84%A0%EC%A1%B1')

In [36]:
for list_num in tqdm(range(1,n+1)):
    # print(list_num)

    # 파일선택
    driver.implicitly_wait(10)
    driver.find_element(By.CSS_SELECTOR, '#divContent > div > div.rightContent.wd756 > div > div.srchResultW > div:nth-child(1) > div.resultTop1 > label > span').click()
    # 내보내기
    driver.implicitly_wait(60)
    driver.find_element(By.CSS_SELECTOR, '#divContent > div > div.rightContent.wd756 > div > div.srchResultW > div:nth-child(1) > div.resultTop1 > ul > li:nth-child(1) > a').click()    # 팝업창 컨트롤
    driver.implicitly_wait(10)
    driver.switch_to.window(driver.window_handles[-1])
    # Excel 저장 선택
    driver.implicitly_wait(20)
    driver.find_element(By.CSS_SELECTOR, '#wrap > form > div > div.popCont > div:nth-child(1) > div > ul > li:nth-child(3) > label').click()
    # Excel 상세정보 선택
    driver.implicitly_wait(10)
    driver.find_element(By.CSS_SELECTOR, '#riss_gubun > ul > li:nth-child(2) > label').click()
    # 데이터 표시양식 '가로' 설정
    driver.implicitly_wait(10)
    driver.find_element(By.CSS_SELECTOR, '#excel_gubun > ul > li:nth-child(2) > label').click()
    # 내보내기 클릭
    driver.implicitly_wait(10)
    driver.find_element(By.CSS_SELECTOR, '#riss_gubun > div.btnBunch > a.btnType1').click()
    
    # 다운로드 완료될 때까지 대기
    driver.implicitly_wait(10)
    try:
        filename = max([download_path + '//' + f for f in os.listdir(download_path) if f=='myCabinetExcelData.xls'],key=os.path.getctime)
    except ValueError as e:
        time.sleep(1)

    # 팝업창 닫기
    driver.implicitly_wait(5)
    driver.close()
    # 다시 메인 화면으로 스위치
    driver.implicitly_wait(5)
    driver.switch_to.window(driver.window_handles[0])

    # 방금 다운 받은 파일 이름 바꾸기 -> 중복방지
    time.sleep(5)
    try:
        filename = max([download_path + '//' + f for f in os.listdir(download_path) if f=='myCabinetExcelData.xls'],key=os.path.getctime)
        time.sleep(0.5)
        shutil.move(filename,os.path.join(download_path,f"{keyword}_{list_num}.xls")) ## 파일 이름 + num 설정
    except ValueError:
        time.sleep(0.5)
    except FileNotFoundError:
        time.sleep(1)

    # 다음 페이지 이동
    time.sleep(1)
    if list_num!=n:
        if (list_num)%10!=0:
            for i in range(10):
            # 다음페이지 클릭
                try:
                    driver.implicitly_wait(10)
                    if(list_num+1)%10!=0:
                        # 10페이지 이하 일때
                        if list_num<10:
                            driver.find_element(By.CSS_SELECTOR, f'#divContent > div > div.rightContent.wd756 > div > div.paging > a:nth-child({((list_num+1)%10)+1})').click()
                        else:
                            driver.find_element(By.CSS_SELECTOR, f'#divContent > div > div.rightContent.wd756 > div > div.paging > a:nth-child({((list_num+1)%10)+2})').click()
                    else:
                        if list_num<10:
                            driver.find_element(By.CSS_SELECTOR, '#divContent > div > div.rightContent.wd756 > div > div.paging > a:nth-child(11)').click()
                        else:
                            driver.find_element(By.CSS_SELECTOR, '#divContent > div > div.rightContent.wd756 > div > div.paging > a:nth-child(12)').click()
                except NoSuchElementException as e:
                    print('retry in 1s.')
                    time.sleep(1)
                else:
                    break
        else:
            # 현재 10 * n 페이지 일때는 다음 화살표 클릭
            driver.implicitly_wait(10)
            # #divContent > div > div.rightContent.wd756 > div > div.paging > a.next1
            driver.find_element(By.CSS_SELECTOR, '#divContent > div > div.rightContent.wd756 > div > div.paging > a.next1').click()
            print('\n next tab')
            # else:
            #     # driver.find_element_by_css_selector('#level4_mainContent > form > div.tabWarp.page > div.tabCont > div > div.passing > span:nth-child(4) > a > img').click()
            #     driver.find_element_by_css_selector('#divContent > div.rightContent > div > div.paging > a.next2').click()
            #     print('next tab')
            #list_num += 1

100%|██████████| 2/2 [00:21<00:00, 10.79s/it]
